In [6]:
!pip install pyngrok

In [7]:
from flask import Flask
from pyngrok import ngrok

In [8]:
port_no = 5000

In [ ]:
from flask import Flask, request, jsonify
import requests
from PIL import Image
import base64
import torch
from transformers import DetrForObjectDetection, DetrImageProcessor, AutoImageProcessor, AutoModelForImageClassification
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from io import BytesIO
import io

app = Flask(__name__)
ngrok.set_auth_token("2dnR9cObpNxE807tfn8rwWJip20_3vC3MByiiEwSzdZ591TW9")
public_url =  ngrok.connect(port_no).public_url

@app.route("/")
def home():
    return """
   <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Your Project Homepage</title>
    <style>
        /* Basic styling */
        body {
            font-family: Arial, sans-serif;
            background-color: #f9f9f9;
            margin: 0;
            padding: 0;
        }

        .header {
            background-color: #333;
            color: #fff;
            padding: 1rem;
            text-align: center;
        }

        .logo {
            font-size: 1.5rem;
            text-decoration: none;
            color: #fff;
        }

        .nav-items a {
            color: #fff;
            text-decoration: none;
            margin-right: 1rem;
        }

        .intro {
            text-align: center;
            padding: 2rem;
            background-color: #fff;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }

        h1 {
            font-size: 2rem;
            margin-bottom: 1rem;
        }

        p {
            font-size: 1rem;
            color: #777;
        }

        button {
            background-color: #333;
            color: #fff;
            border: none;
            padding: 0.5rem 1rem;
            cursor: pointer;
            border-radius: 5px;
        }

        footer {
            background-color: #333;
            color: #fff;
            text-align: center;
            padding: 1rem;
        }

        .social-media a {
            color: #fff;
            margin: 0 0.5rem;
            font-size: 1.5rem;
        }

        /* Additional styling for images */
        .image-container {
            display: flex;
            justify-content: center;
            margin-top: 2rem;
        }

        .image-container img {
            width: 300px;
            height: auto;
            margin: 0 1rem;
            cursor: pointer;
            transition: transform 0.3s ease;
        }

        .image-container img:hover {
            transform: scale(1.1);
        }

        /* Loader styling */
        .loader {
            border: 8px solid #f3f3f3; /* Light grey */
            border-top: 8px solid #3498db; /* Blue */
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 2s linear infinite;
            display: none; /* Initially hidden */
            position: fixed;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        button.predict-btn:hover {
            background-color: #2980b9;
        }
       /* Predict button styling */
        .predict-btn-container {
            display: flex;
            justify-content: center;
            margin-top: 1rem;
        }

        .predict-btn:hover {
            background-color: #2980b9;
        }

        .predict-btn {
            background-color: #3498db;
            color: #fff;
            border: none;
            padding: 0.5rem 1rem;
            cursor: pointer;
            border-radius: 5px;
            font-size: 1rem;
            transition: background-color 0.3s ease;
        }

        .file-input {
            display: none;
        }
    </style>
</head>
<body>
    <header class="header">
        <a href="#" class="logo">Emotion Detection</a>
        <nav class="nav-items">
            <a href="#">Home</a>
            <a href="#">About</a>
            <a href="#">Contact</a>
        </nav>
    </header>

    <main>
        <div class="intro">
            <h1>Emotion Detection</h1>
            <p>This project aims to detect emotions in images using advanced machine learning models. Simply upload an image, and our system will emotions, and display the results in real-time.</p>

            <div class="predict-btn-container">
            <button class="predict-btn" onclick="predict()">Predict</button>
        </div>
        </div>

        <div class="image-container">
            <img id="selectedImage1" src="https://mdbootstrap.com/img/Photos/Others/placeholder.jpg" alt="example placeholder" onclick="document.getElementById('customFile1').click()">
             <div></div>
            <img id="selectedImage2" src="https://mdbootstrap.com/img/Photos/Others/placeholder.jpg" alt="example placeholder" onclick="document.getElementById('customFile2').click()">
        </div>



        <input type="file" class="file-input" id="customFile1" onchange="displaySelectedImage(event, 'selectedImage1')">
        <input type="file" class="file-input" id="customFile2" onchange="displaySelectedImage(event, 'selectedImage2')">

        <div id="loader" class="loader"></div>
    </main>

    <footer>
        <p>&copy; 2024 Your Project. All rights reserved.</p>
        <div class="social-media">
            <a href="#"><i class="fab fa-facebook"></i></a>
            <a href="#"><i class="fab fa-twitter"></i></a>
            <a href="#"><i class="fab fa-linkedin"></i></a>
        </div>
    </footer>

    <script>
        function displaySelectedImage(event, elementId) {
            const selectedImage = document.getElementById(elementId);
            const fileInput = event.target;

            if (fileInput.files && fileInput.files[0]) {
                const reader = new FileReader();

                reader.onload = function(e) {
                    selectedImage.src = e.target.result;
                };

                reader.readAsDataURL(fileInput.files[0]);
            }
        }

        async function predict() {
            // Show loader
            showLoader();

            try {
                const selectedImage1 = document.getElementById('selectedImage1');
                const imageData = selectedImage1.src.split(',')[1]; // Extract base64-encoded image data

                const response = await fetch('/predict', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ image1: imageData })
                });

                if (!response.ok) {
                    throw new Error('Failed to predict');
                }

                const data = await response.json();

                // Update the src attribute of the second image element
                const selectedImage2 = document.getElementById('selectedImage2');
                selectedImage2.src = 'data:image;base64,' + data.image2;

                // Hide loader
                hideLoader();
            } catch (error) {
                console.error('Prediction error:', error);
                // Handle error
                // Hide loader
                hideLoader();
            }
        }

        function showLoader() {
            // Show loader element
            const loader = document.getElementById('loader');
            loader.style.display = 'block';
        }

        function hideLoader() {
            // Hide loader element
            const loader = document.getElementById('loader');
            loader.style.display = 'none';
        }
    </script>
</body>
</html>

    """

def predict_emotions(image_base64):
    try:
        image_data = base64.b64decode(image_base64)
        image = Image.open(BytesIO(image_data))


        object_detection_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
        object_detection_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")


        inputs = object_detection_processor(images=image, return_tensors="pt")
        object_outputs = object_detection_model(**inputs)


        target_sizes = torch.tensor([image.size[::-1]])
        object_results = object_detection_processor.post_process_object_detection(object_outputs, target_sizes=target_sizes, threshold=0.9)[0]


        model_path = "/content/drive/MyDrive/CharbelMATTAR_LU_WebDev/Project/Model"
        emotion_processor = AutoImageProcessor.from_pretrained(model_path)
        model_path = "/content/drive/MyDrive/CharbelMATTAR_LU_WebDev/Project/Model"
        emotion_model = AutoModelForImageClassification.from_pretrained(model_path)


        person_counter = 0


        fig, ax = plt.subplots(1)
        ax.imshow(image)

        for score, label, box in zip(object_results["scores"], object_results["labels"], object_results["boxes"]):
            box = [round(i, 2) for i in box.tolist()]
            class_name = object_detection_model.config.id2label[label.item()]
            if class_name == "person":
                person_counter += 1

                roi = image.crop((box[0], box[1], box[2], box[3]))

                emotion_inputs = emotion_processor(roi, return_tensors="pt")

                emotion_outputs = emotion_model(**emotion_inputs)

                predicted_probabilities = torch.softmax(emotion_outputs.logits, dim=1)

                predicted_emotion_index = torch.argmax(predicted_probabilities, dim=1)
                predicted_emotion_class = predicted_emotion_index.item()

                emotion_labels = ['Ahegao', 'Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']
                predicted_emotion = emotion_labels[predicted_emotion_class]

                label_string = f"Person {person_counter}: {round(score.item(), 3)}, Emotion: {predicted_emotion}"

                rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                         linewidth=1, edgecolor='blue', facecolor='none')
                ax.add_patch(rect)
                ax.text(box[0], box[1] - 5, label_string, color='blue')
            elif class_name == "cell phone":
                label_string = f"{class_name}: {round(score.item(), 3)}"

                ax.text(box[0], box[1] - 5, label_string, color='green')


        plt.axis('off')
        plt.tight_layout()


        output_image_buffer = BytesIO()
        plt.savefig(output_image_buffer, format='png', bbox_inches='tight', pad_inches=0)  # Save with tight bounding box
        output_image_buffer.seek(0)

        # Convert the image buffer to base64 string
        output_image_base64 = base64.b64encode(output_image_buffer.getvalue()).decode('utf-8')

        return output_image_base64



    except Exception as e:
        print("Error:", str(e))
        return None

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Retrieve image data from the request
        image_base64 = request.json['image1']


        predicted_image = predict_emotions(image_base64)

        if predicted_image:
            print('222')
            # Decode the predicted image from base64
            predicted_image_bytes = base64.b64decode(predicted_image)
            predicted_image = Image.open(BytesIO(predicted_image_bytes))

            plt.imshow(predicted_image)
            plt.axis('off')
            plt.tight_layout()
            plt.show()


            predicted_image_base64 = base64.b64encode(predicted_image_bytes).decode('utf-8')

        if predicted_image:
            return jsonify({'image2': predicted_image_base64})
        else:
            return jsonify({'error': 'Failed to predict'})

    except Exception as e:
        return jsonify({'error': str(e)})



print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)